In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
import wandb

# Load the dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
# Normalize images
train_images, test_images = train_images / 255.0, test_images / 255.0
# Define class names
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
#intialize wanddb
wandb.init(project="deep_learning",name='mnist_dataset')
# Select one image per class
samples_per_class = {}
for img, label in zip(train_images, train_labels):
    if label not in samples_per_class:
        samples_per_class[label] = img
    if len(samples_per_class) == len(class_names):
        break
wandb.log({"Fashion-MNIST Sample Images": [wandb.Image(img, caption=class_names[label]) for label, img in samples_per_class.items()]})

# Plot images
fig, axes = plt.subplots(2, 5, figsize=(10, 5))
fig.suptitle("Fashion-MNIST Sample Images", fontsize=14)

for ax, (label, image) in zip(axes.flat, samples_per_class.items()):
    ax.imshow(image, cmap='gray')
    ax.set_title(class_names[label])
    ax.axis('off')

plt.tight_layout()
plt.show()
wandb.finish()

In [ ]:
#flatten the images
train_images = train_images.reshape(train_images.shape[0], -1)
test_images = test_images.reshape(test_images.shape[0], -1)

In [ ]:
import numpy as np

# Helper Functions
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Subtract max for numerical stability
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

def relu(z):
    return np.maximum(0, z)

def relu_derivative(z):
    return (z > 0).astype(float)


def cross_entropy_loss(y_true, y_pred):
    n_samples = y_true.shape[0]
    logp = -np.log(y_pred[range(n_samples), y_true])
    return np.sum(logp) / n_samples
# Neural Network Class
class NeuralNetwork:
    def __init__(self, layer_sizes):
        self.layer_sizes = layer_sizes
        self.weights = []
        self.biases = []
        self._initialize_parameters()

    def _initialize_parameters(self):
        """Initialize weights and biases for all layers."""
        for i in range(len(self.layer_sizes) - 1):
            fan_in = self.layer_sizes[i]
            fan_out = self.layer_sizes[i + 1]
            # He initialization for hidden layers, Xavier for output layer
            if i < len(self.layer_sizes) - 2:  # Hidden layers use sigmoid
                scale = np.sqrt(2 / fan_in)
            else:  # Output layer uses softmax
                scale = np.sqrt(1 / fan_in)
            # Weight matrix: (fan_in, fan_out)
            weight = np.random.randn(fan_in, fan_out) * scale
            # Bias vector: (fan_out,)
            bias = np.zeros(fan_out)
            self.weights.append(weight)
            self.biases.append(bias)

    def forward(self, x):
        a = x  # Activation starts as the input
        # Process hidden layers
        for l in range(len(self.weights) - 1):
            z = np.dot(a, self.weights[l]) + self.biases[l]
            a = sigmoid(z)
        # Process output layer
        z = np.dot(a, self.weights[-1]) + self.biases[-1]
        output = softmax(z)
        return output

# Example Usage
if __name__ == "__main__":
    # Create a network: 784 input -> 256 hidden -> 128 hidden -> 10 output
    nn = NeuralNetwork([784, 256, 128, 10])
    # Simulate 5 input samples (e.g., Fashion-MNIST images)
    x = np.random.randn(5, 784)
    # Get probabilities
    probs = nn.forward(x)
    # Print probabilities for each sample
    for i, prob in enumerate(probs):
        print(f"Sample {i+1} probabilities: {prob}")

In [ ]:
def to_one_hot(labels, num_classes=10):
    """Convert labels to one-hot encoding."""
    return np.eye(num_classes)[labels]

# Base Optimizer Class
class Optimizer:
    """Abstract base class for optimizers."""
    def update(self, weights, biases, grads_w, grads_b):
        raise NotImplementedError("Subclasses must implement this method.")

# SGD Optimizer
class SGD(Optimizer):
    def __init__(self, learning_rate=0.01):
        self.learning_rate = learning_rate

    def update(self, weights, biases, grads_w, grads_b):
        for i in range(len(weights)):
            weights[i] -= self.learning_rate * grads_w[i]
            biases[i] -= self.learning_rate * grads_b[i]



In [ ]:
# Momentum Optimizer
class Momentum(Optimizer):
    def __init__(self, learning_rate=0.01, momentum=0.9):
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.velocity_w = None
        self.velocity_b = None

    def update(self, weights, biases, grads_w, grads_b):
        if self.velocity_w is None:
            self.velocity_w = [np.zeros_like(w) for w in weights]
            self.velocity_b = [np.zeros_like(b) for b in biases]
        for i in range(len(weights)):
            self.velocity_w[i] = self.momentum * self.velocity_w[i] + self.learning_rate * grads_w[i]
            self.velocity_b[i] = self.momentum * self.velocity_b[i] + self.learning_rate * grads_b[i]
            weights[i] -= self.velocity_w[i]
            biases[i] -= self.velocity_b[i]

In [ ]:
# Nesterov Optimizer
class Nesterov(Optimizer):
    def __init__(self, learning_rate=0.01, momentum=0.9):
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.velocity_w = None
        self.velocity_b = None

    def update(self, weights, biases, grads_w, grads_b):
        if self.velocity_w is None:
            self.velocity_w = [np.zeros_like(w) for w in weights]
            self.velocity_b = [np.zeros_like(b) for b in biases]
        for i in range(len(weights)):
            # Simplified Nesterov: update with current gradients (approximation)
            self.velocity_w[i] = self.momentum * self.velocity_w[i] + self.learning_rate * grads_w[i]
            self.velocity_b[i] = self.momentum * self.velocity_b[i] + self.learning_rate * grads_b[i]
            weights[i] -= self.momentum * self.velocity_w[i]
            biases[i] -= self.momentum * self.velocity_b[i]

In [ ]:
# RMSProp Optimizer
class RMSProp(Optimizer):
    def __init__(self, learning_rate=0.001, beta=0.9, epsilon=1e-8):
        self.learning_rate = learning_rate
        self.beta = beta
        self.epsilon = epsilon
        self.s_w = None
        self.s_b = None

    def update(self, weights, biases, grads_w, grads_b):
        if self.s_w is None:
            self.s_w = [np.zeros_like(w) for w in weights]
            self.s_b = [np.zeros_like(b) for b in biases]
        for i in range(len(weights)):
            self.s_w[i] = self.beta * self.s_w[i] + (1 - self.beta) * (grads_w[i] ** 2)
            self.s_b[i] = self.beta * self.s_b[i] + (1 - self.beta) * (grads_b[i] ** 2)
            weights[i] -= self.learning_rate * grads_w[i] / (np.sqrt(self.s_w[i] + self.epsilon))
            biases[i] -= self.learning_rate * grads_b[i] / (np.sqrt(self.s_b[i] + self.epsilon))


In [ ]:
# Adam Optimizer
class Adam(Optimizer):
    def __init__(self, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.learning_rate = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m_w = None
        self.m_b = None
        self.v_w = None
        self.v_b = None
        self.t = 0

    def update(self, weights, biases, grads_w, grads_b):
        if self.m_w is None:
            self.m_w = [np.zeros_like(w) for w in weights]
            self.m_b = [np.zeros_like(b) for b in biases]
            self.v_w = [np.zeros_like(w) for w in weights]
            self.v_b = [np.zeros_like(b) for b in biases]
        self.t += 1
        for i in range(len(weights)):
            self.m_w[i] = self.beta1 * self.m_w[i] + (1 - self.beta1) * grads_w[i]
            self.m_b[i] = self.beta1 * self.m_b[i] + (1 - self.beta1) * grads_b[i]
            self.v_w[i] = self.beta2 * self.v_w[i] + (1 - self.beta2) * (grads_w[i] ** 2)
            self.v_b[i] = self.beta2 * self.v_b[i] + (1 - self.beta2) * (grads_b[i] ** 2)
            m_w_hat = self.m_w[i] / (1 - self.beta1 ** self.t)
            m_b_hat = self.m_b[i] / (1 - self.beta1 ** self.t)
            v_w_hat = self.v_w[i] / (1 - self.beta2 ** self.t)
            v_b_hat = self.v_b[i] / (1 - self.beta2 ** self.t)
            weights[i] -= self.learning_rate * m_w_hat / (np.sqrt(v_w_hat) + self.epsilon)
            biases[i] -= self.learning_rate * m_b_hat / (np.sqrt(v_b_hat) + self.epsilon)


In [ ]:
# Nadam Optimizer
class Nadam(Optimizer):
    def __init__(self, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.learning_rate = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m_w = None
        self.m_b = None
        self.v_w = None
        self.v_b = None
        self.t = 0

    def update(self, weights, biases, grads_w, grads_b):
        if self.m_w is None:
            self.m_w = [np.zeros_like(w) for w in weights]
            self.m_b = [np.zeros_like(b) for b in biases]
            self.v_w = [np.zeros_like(w) for w in weights]
            self.v_b = [np.zeros_like(b) for b in biases]
        self.t += 1
        for i in range(len(weights)):
            self.m_w[i] = self.beta1 * self.m_w[i] + (1 - self.beta1) * grads_w[i]
            self.m_b[i] = self.beta1 * self.m_b[i] + (1 - self.beta1) * grads_b[i]
            self.v_w[i] = self.beta2 * self.v_w[i] + (1 - self.beta2) * (grads_w[i] ** 2)
            self.v_b[i] = self.beta2 * self.v_b[i] + (1 - self.beta2) * (grads_b[i] ** 2)
            m_w_hat = self.m_w[i] / (1 - self.beta1 ** self.t)
            m_b_hat = self.m_b[i] / (1 - self.beta1 ** self.t)
            v_w_hat = self.v_w[i] / (1 - self.beta2 ** self.t)
            v_b_hat = self.v_b[i] / (1 - self.beta2 ** self.t)
            m_w_nesterov = self.beta1 * m_w_hat + (1 - self.beta1) * grads_w[i] / (1 - self.beta1 ** self.t)
            m_b_nesterov = self.beta1 * m_b_hat + (1 - self.beta1) * grads_b[i] / (1 - self.beta1 ** self.t)
            weights[i] -= self.learning_rate * m_w_nesterov / (np.sqrt(v_w_hat) + self.epsilon)
            biases[i] -= self.learning_rate * m_b_nesterov / (np.sqrt(v_b_hat) + self.epsilon)
